# Inheritance

How the Python interpreter looks up attributes:

1. it checks for an instance variable or method.  If not found, then ...
2. it checks for its class' variable/method.  If not found, then ...
3. it looks for a class variable in the parent class.  If not found, then ...
4. this process goes on until the last ancestor is reached, at which point Python will signal an error.

## Tamagotchi Pet super class

In [250]:
from random import randrange # returns integer

class Pet:
    """Class Pet creates a Tamagotchi"""
        
    boredom_decrement = 4
    hunger_decrement = 6
    boredom_threshold = 5
    hunger_threshold = 10
    sounds = ['Mrrp']
    
    def __init__(self, name = 'Kitty'):
        self.name = name
        self.hunger = randrange(self.hunger_threshold) #random
        self.boredom = randrange(self.boredom_threshold) #random
        self.sounds = self.sounds[:] # copy the class attribute sounds so we can make changes per pet
    
    def __str__(self):
        state = 'I\'m {}. '.format(self.name)
        state += 'I feel {}.'.format(self.mood())
        return state
    
    def clock_tick(self):
        self.boredom += 1
        self.hunger += 1
    
    def reduce_boredom(self):
        self.boredom = max(self.boredom - self.boredom_decrement, 0)
    
    def reduce_hunger(self):
        self.hunger = max(self.hunger - self.hunger_decrement, 0)
    
    def teach(self, word):
        self.sounds.append(word)
        self.reduce_boredom()
        
    def hi(self):
        sound = self.sounds[randrange(0,len(self.sounds))]
        print(sound)
        self.reduce_boredom()
    
    def feed(self):
        self.reduce_hunger()
    
    def mood(self):
        if self.hunger <= self.hunger_threshold and self.boredom <= self.boredom_threshold:
            return 'happy'
        elif self.hunger > self.hunger_threshold:
            return 'hungry'
        else:
            return 'bored'
        
    def check_stats(self):
        print('Boredom: {}.  Hunger: {}.'.format(self.boredom, self.hunger))
    

In [251]:
wulfric = Pet("Wulfric")
wulfric.teach("Rowl")
wulfric.sounds

['Mrrp', 'Rowl']

## Create sub classes
### Cat class
* inherits from `Pet` class
* has a different starting sound (`"Meow"`)
* has a new `chase_rats()` method

In [252]:
class Cat(Pet):
    sounds = ['Meow']
    def chase_rats(self):
        print('What are you doing, Pinky? Taking over the world?!')

In [253]:
percy = Cat("Percy")
print(percy)
percy.hi()
percy.chase_rats()
percy.teach("Brrrrruuup")

I'm Percy. I feel happy.
Meow
What are you doing, Pinky? Taking over the world?!


### Cheshire class

* inherit from `Cat` (which inherits from `Pet`)
* has a method called `smile()`

In [254]:
class Cheshire(Cat):
    def smile(self):
        print(":D :D :D")

In [255]:
kiefer = Cheshire('Kiefer')
print(kiefer)
kiefer.hi()
kiefer.chase_rats()
kiefer.teach("Muchness")
kiefer.smile()

I'm Kiefer. I feel happy.
Meow
What are you doing, Pinky? Taking over the world?!
:D :D :D


## Are subclasses instances? (answer: no)

Use `issubclass(class_to_check, super_class)` to check if a class is a subclass of another class.

Use `isinstance(inst_to_check, class)`to check of an object is an instance.

Results:
* a progeny class is a __subclass__ of all its __ancester__ class(es).
* an instance is an __instance__ of all its __ancester__ class(es).
* a __subclass__ is __not__ an __instance__.

Creating a subclass doesn't run the master Class' `__init__`.  When an instance of a subclass is created, the subclass' `__init__` is run.  If it doesn't have an `__init__` method, then the master Class' `__init__` is run instead.

Therefore it is not necessary for a subclass to have an `__init__` unless it will have additional instance variables that the master Class doesn't have.

A subclass introducing either of the following does not require a new `__init__`:
* redefining an instance variable's value
* new or redefined methods

In [279]:
## check subclasses:

print('Is Cat a subclass of Pet?', issubclass(Cat, Pet))
print('Is Cheshire a subclass of Pet?', issubclass(Cheshire, Pet))


Is Cat a subclass of Pet? True
Is Cheshire a subclass of Pet? True


In [282]:
## check instances:

print('Is percy an instance of Cat?', isinstance(percy, Cat))
print('Is percy also an instance of Pet?', isinstance(percy, Pet))
print()
print('Is the class Cat and instance of Pet?', isinstance(Cat, Pet))

Is percy an instance of Cat? True
Is percy also an instance of Pet? True

Is the class Cat and instance of Pet? False


## Class variables vs instance variables

[Excellent article on Medium](https://medium.com/python-features/class-vs-instance-variables-8d452e9abcbd)

### class variables

Declared inside the class definition (but outside any of the instance methods). They are not tied to any particular object of the class, hence __shared across all__ the objects of the class. __Modifying__ a class variable __affects all__ objects instance at the same time.

### instance variables

Declared __inside the constructor method__ of class (the `__init__` method). They are tied to the __particular object instance__ of the class, hence the contents of an instance variable are completely __independent__ from one object instance to the other.

### variable examples

No new class or instance variables were introduced in `Cat` or `Cheshire`.

`Cat` did re-define the instance variable `sounds` from `['Mrrp']` to `['Meow']`.

All instances created from `Pet`, `Cat`, and `Cheshire`, have the same instance variables since no new instance variables were introduced in `Cat` or `Cheshire`:
* name
* hunger
* boredom
* sounds (`sounds` was changed from a Class variable into an instance variable in `Pet`'s `__init__` )

They also all have the same class variables:
* boredom_decrement
* hunger_decrement
* boredom_threshold
* hunger_threshold
* ~~_sounds_~~ (`sounds` was changed into an instance variable in `Pet`'s `__init__`)

## my method to find instance and class variables

I wrote this method to help me understand these variable categories better.

In [256]:
def inst_and_class_vars(class_or_instance):
    instance_vars = [
        k for k, v in vars(class_or_instance).items()
        if not callable(v)
        and not k.startswith("__")
    ]
    class_vars = [
        cl for cl in dir(class_or_instance)
        if str(type(getattr(class_or_instance, cl))) != "<class 'method'>"
        and str(type(getattr(class_or_instance, cl))) != "<class 'function'>"
        and not cl.startswith("__")
        and cl not in instance_vars
    ]
    print('Instance variables:')
    print(instance_vars)
    print('')
    print('Class variables')
    print(class_vars)

### variables for my instances

In [258]:
inst_and_class_vars(wulfric)

Instance variables:
['name', 'hunger', 'boredom', 'sounds']

Class variables
['boredom_decrement', 'boredom_threshold', 'hunger_decrement', 'hunger_threshold']


In [260]:
inst_and_class_vars(percy)

Instance variables:
['name', 'hunger', 'boredom', 'sounds']

Class variables
['boredom_decrement', 'boredom_threshold', 'hunger_decrement', 'hunger_threshold']


In [262]:
inst_and_class_vars(kiefer)

Instance variables:
['name', 'hunger', 'boredom', 'sounds']

Class variables
['boredom_decrement', 'boredom_threshold', 'hunger_decrement', 'hunger_threshold']


### how sound (instance variable) value changes

In [285]:
# Pet vs wulfric
print('Pet class sounds:', Pet.sounds)
print('Pet instance, wulfric sounds after training:', wulfric.sounds)

Pet class sounds: ['Mrrp']
Pet instance, wulfric sounds after training: ['Mrrp', 'Rowl']


In [286]:
# Cat vs percy
print('Cat class sounds:', Cat.sounds)
print('Cat instance, percy sounds after training:', percy.sounds)

Cat class sounds: ['Meow']
Cat instance, percy sounds after training: ['Meow', 'Brrrrruuup']


In [287]:
# Cheshire vs kiefer
print('Cheshire class sounds:', Cheshire.sounds)
print('Cheshier instance, kiefer sounds after training:', kiefer.sounds)

Cheshire class sounds: ['Meow']
Cheshier instance, kiefer sounds after training: ['Meow', 'Muchness']


## Overriding methods

### sub classes: Dog and Cat (rewritten)

* `Dog` -- *always* happy unless bored *and* hungry
    * happy
    * bored *and* hungry
* `Cat` -- *only* happy if fed *and* not bored *and* not annoyed
    * hungry (< threshold)
    * grumpy; leave me alone (bored < 2)
    * bored (< threshold)
    * randomly annoyed (1/2 of the time)
    * happy

In [388]:
class Cat(Pet):
    """Cat class as a subclass of Pet is hard to make happy"""
    
    sounds = ['Meow']
    
    def mood(self):
        if self.hunger > self.hunger_threshold:
            return "hungry"
        elif self.boredom < 2:
            return "grumpy; leave me alone"
        elif self.boredom > self.boredom_threshold:
            return "bored"
        elif randrange(2) == 0: # 0 or 1
            return "randomly annoyed"
        else:
            return "happy"

In [313]:
class Dog(Pet):
    """Dog class as subclass of Pet is almost always happy"""
    
    sounds = ['Woof']
    
    def mood(self):
        if (self.hunger > self.hunger_threshold) and (self.boredom > self.boredom_threshold):
            return "bored and hungry"
        else:
            return "happy"

In [314]:
fluffy = Cat('Fluffy')
astro = Dog('Astro')

In [316]:
fluffy.boredom = 1
fluffy.mood()

'grumpy; leave me alone'

In [320]:
fluffy.boredom = 3
for i in range(10):
    print(fluffy.mood())

randomly annoyed
happy
happy
happy
happy
randomly annoyed
randomly annoyed
randomly annoyed
randomly annoyed
happy


In [321]:
astro.mood()

'happy'

## Invoking parent Class' method

call parent's method when redefining that method in the subclass

### using `Pet`'s `feed` method in `Dog`'s `feed` definition
* should still decrease hunger (`Pet`'s `feed` method)
* but should also say "Aarf! Thanks"

In [322]:
class Dog(Pet):
    """Dog class as subclass of Pet is almost always happy"""
    
    sounds = ['Woof']
    
    def mood(self):
        if (self.hunger > self.hunger_threshold) and (self.boredom > self.boredom_threshold):
            return "bored and hungry"
        else:
            return "happy"
        
    def feed(self): #redefining the feed method
        Pet.feed(self) # first run Pet's feed method
        print("Aarf!  Thanks!") # custom for Dog
        

In [328]:
ginger = Dog("Ginger")
print('Ginger\'s hunger is:', ginger.hunger)
ginger.feed()
print('Ginger\'s hunger is:', ginger.hunger)

Ginger's hunger is: 8
Aarf!  Thanks!
Ginger's hunger is: 2


### `super()` is a much more Pythonic way
* don't need to remember which Class defines the method you want to call
* crawl's up the ancestor classes until it finds the method
* __NOTE:__ `super()` can be used as a __parameterless__ method since `super()` was defined with `self` as the default.  No need to pass `self` into `super().feed()`.
* you can also use `super(<specific ancestor class>, self)` if you want to have more control on which ancester Class you want to use.

In [335]:
class Dog(Pet):
    """Dog class as subclass of Pet is almost always happy"""
    
    sounds = ['Woof']
    
    def mood(self):
        if (self.hunger > self.hunger_threshold) and (self.boredom > self.boredom_threshold):
            return "bored and hungry"
        else:
            return "happy"
        
    def feed(self): #redefining the feed method
        super().feed() # first run Pet's feed method
        print("Aarf!  Thanks!") # custom for Dog
        

In [336]:
ginger = Dog("Ginger")
print('Ginger\'s hunger is:', ginger.hunger)
ginger.feed()
print('Ginger\'s hunger is:', ginger.hunger)

Ginger's hunger is: 1
Aarf!  Thanks!
Ginger's hunger is: 0


### example running ancestor Class `__init__()`
let's say we want to:
* redefine initial `sound` value
* `__init__` with a new instance variable: `chirp_number`
* ensure `sound` does not become a class variable
* ensure the other `Pet` instance variables, `hunger`, `boredom`, `name` are retained

In [384]:
class Bird(Pet):
    sounds = ['Chirp'] # would become a class variable unless redefined in __init__
    
    def __init__(self, name='Kitty', chirp_number=2):
        super().__init__(name) # name, hunger, boredom, sounds
        # Pet.__init__(self, name) # the non-Pythonic way
        self.chirp_number = chirp_number
        
    def hi(self):
        for i in range(self.chirp_number):
            print(self.sounds[randrange(0,len(self.sounds))])
        self.reduce_boredom()

In [386]:
tweety = Bird("Tweety", chirp_number=5)
print(tweety.name, 'says:')
tweety.teach("I tawt I taw a putty cat")
tweety.hi()

Tweety says:
Chirp
I tawt I taw a putty cat
Chirp
Chirp
I tawt I taw a putty cat
